# 02 RAG | 03 Embeddings | 01 Text Embeddings

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The Azure OpenAI client from the Azure.AI.OpenAI NuGet package offers a unified interface to interact with various specialized clients, each designed to handle specific tasks. 

These include clients for chat completion, embeddings, and the agent framework, among others, enabling developers to easily integrate and utilize OpenAI's capabilities for generating text, understanding context, and building intelligent applications within the Azure ecosystem.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Embeddings;
using DotNetEnv; 
using System.IO;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "assets");

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

EmbeddingClient embeddingClient = azureOpenAIClient.GetEmbeddingClient(embeddingDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"EmbeddingClient created...");

## Step 2: Create Embeddings

In the code cell two embeddings or vectors are calculated using the deployed OpenAI model (text-embedding-ada-002):
- The file `Path.Combine(assetsFolder, "Embedding", "TheUntouchableAdolescents_Part01.txt");` is part of a free e-book downloaded from the [Project Gutenberg](www.gutenberg.com)
- A hard coded string `"Did Shreve gave a bitter laugh, thin and short?"` will be also transformed into an embedding or vector representation. You can find the phrase at the end of the free e-book.

To calculate the embeddings or vectors the method `CreateEmbeddingAsync()` from the `EmbeddingsClient` instance is be used. 

Both embeddings will be used in further samples to perform e.g. a vector distance calculation.

In [ ]:

using System.ClientModel; 

// Vectorize AKS (Azure Kubernetes Service) information from Wikipedia
string documentationPage = Path.Combine(assetsFolder,"Embedding", "WikiAks.txt");
string textToBeVectorized = File.ReadAllText(documentationPage);

ClientResult<Embedding> clientResultAKS =  
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);
        
Console.WriteLine($"Vector wiki info AKS created...");

// Vectorize Super Bowl information from Wikipedia
documentationPage = Path.Combine(assetsFolder,"Embedding", "WikiSuperBowl2024.txt");
textToBeVectorized = File.ReadAllText(documentationPage);

ClientResult<Embedding> clientResultSuperBowl =  
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);

Console.WriteLine($"Vector wiki info Super Bowl created...");

// Vectorize a question
textToBeVectorized = "Who won the Super Bowl in 2024";
ClientResult<Embedding> clientResultQuestion = 
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);

Console.WriteLine($"Vector question Super Bowl created...");


**Expected output:**

Vector wiki info AKS created...

Vector wiki info Super Bowl created...

Vector question Super Bowl created...

## Step 3: Vector storage

All three vectors are stored in a txt file for further analyze and/or usage.

In [ ]:
using System.Linq;

string wikiAKSFileName = "../Assets/Embedding/WikiAksVector.txt";
string wikiSuperBowlFileName = "../Assets/Embedding/WikiSuperBowlVector.txt";
string queryVectorFileName = "../Assets/Embedding/QueryVector.txt";

await File.WriteAllLinesAsync(
    wikiAKSFileName, 
    clientResultAKS.Value.Vector.ToArray().Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    wikiSuperBowlFileName, 
    clientResultSuperBowl.Value.Vector.ToArray().Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    queryVectorFileName, 
    clientResultQuestion.Value.Vector.ToArray().Select(vector => vector.ToString())
);

Console.WriteLine($"AKS vector stored in {wikiAKSFileName}...");
Console.WriteLine($"Super Bowl vector stored in {wikiSuperBowlFileName}...");
Console.WriteLine($"Query Vector stored in {queryVectorFileName}...");


**Expected result:**

AKS vector stored in ../Assets/Embedding/WikiAksVector.txt...

Super Bowl vector stored in ../Assets/Embedding/WikiSuperBowlVector.txt...

Query Vector stored in ../Assets/Embedding/QueryVector.txt...